In [2]:
import random
import operator
import copy
import pprint
import numpy as np
import pandas as pd
# import gp
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import KFold
import deap.gp as gp
from deap import creator, base, tools, algorithms
from deap.gp import cxOnePoint as cx_simple
from deap.gp import PrimitiveSet
from data import get_embeddings

#### Set parameters

In [3]:
WORD2VEC = "word2vec"
GLOVE = "glove"
FASTTEXT = "fasttext"
CX_RANDOM = 0
CX_SIMPLE = 1
CX_UNIFORM = 2
CX_FAIR = 3
CX_ONEPOINT = 4

#### Fuctions

In [4]:
# Arithmetic operators
def protected_div(x, y):
    mask = y == 0
    safe_y = np.where(mask, 1, y)
    return np.where(mask, 1, x / safe_y)


def protected_sqrt(x):
    sign = np.sign(x)
    x = np.abs(x)
    return np.sqrt(x) * sign

In [5]:
# Crossover method
def subtree_height(tree, index):
        """
        Calculate the height of the subtree starting at the given index.
        """
        def height(node_index):
            node = tree[node_index]
            if node.arity == 0:  # Leaf node
                return 1
            else:
                return 1 + max(
                    height(child_index)
                    for child_index in range(
                        node_index + 1, node_index + 1 + node.arity
                    )
                )

        return height(index)

def searchSubtree_idx(tree, begin):
        end = begin + 1
        total = tree[begin].arity
        while total > 0:
            total += tree[end].arity - 1
            end += 1
        return begin, end

def cx_uniform(ind1, ind2):
        # No crossover on single node tree
        if (len(ind1) < 2 or len(ind2) < 2):
            return ind1, ind2

        child = type(ind1)([])
        parents = [ind1, ind2]
        flag0, flag1 = 0, 0
        left_0 = parents[0].searchSubtree(1)
        left_1 = parents[1].searchSubtree(1)
        b0, e0 = searchSubtree_idx(parents[0], 1)
        b1, e1 = searchSubtree_idx(parents[1], 1)

        if e0 + 1 < len(parents[0]):
            right_0 = parents[0].searchSubtree(e0 + 1)
            flag0 = 1
        if e1 + 1 < len(parents[1]):
            right_1 = parents[1].searchSubtree(e1 + 1)
            flag1 = 1
        left = [left_0, left_1]
        if flag0 == 1 and flag1 == 1:
            right = [right_0, right_1]
            r_arity = 0
            if parents[0][e0 + 1].arity == parents[1][e1 + 1].arity:
                r_arity = 1
        r = random.randint(0, 1)  # root
        m = 1 - r
        if len(parents[r]) < len(parents[m]):
            # root = parents[r].root
            if flag1 == 0 or flag0 == 0:
                return parents[r], parents[m]
            parents[m][0] = parents[r].root
            m = r
        if flag0 == 1 and flag1 == 1:
            r1 = random.randint(0, 1)  # 左邊
            if parents[r][1] == parents[r1][1]:
                parents[r][left[r]] = parents[r1][left[r1]]
            if r_arity == 1:
                r2 = random.randint(0, 1)
                parents[r][right[r]] = parents[r2][right[r2]]
        else:
            # print("只有一個子點")
            r1 = random.randint(0, 1)
            parents[r][left[r1]] = parents[r1][left[r1]]
        return parents[r], parents[r]

def cx_fair(ind1, ind2):
    """Size fair crossover for two trees.
    :param ind1: First tree participating in the crossover.
    :param ind2: Second tree participating in the crossover.
    :returns: A tuple of two trees.
    """
    # No crossover on single node tree
    if len(ind1) < 2 or len(ind2) < 2:
        return ind1, ind2

    # List all available primitive types in each individual
    types1 = gp.defaultdict(list)
    types2 = gp.defaultdict(list)
    if ind1.root.ret == gp.__type__:
        # Not STGP optimization
        types1[gp.__type__] = list(range(1, len(ind1)))
        types2[gp.__type__] = list(range(1, len(ind2)))
        common_types = [gp.__type__]
    else:
        for idx, node in enumerate(ind1[1:], 1):
            types1[node.ret].append(idx)
        for idx, node in enumerate(ind2[1:], 1):
            types2[node.ret].append(idx)
        common_types = set(types1.keys()).intersection(set(types2.keys()))

    if len(common_types) > 0:
        type_ = random.choice(list(common_types))

    index1 = random.choice(types1[type_])
    height1 = subtree_height(ind1, index1)
    # height = ind1.height

    while True:
        index2 = random.choice(types2[type_])
        height2 = subtree_height(ind2, index2)
        if height2 <= height1:
            # print(f"height1: {height1}, height2: {height2}")
            break
    slice1 = ind1.searchSubtree(index1)
    slice2 = ind2.searchSubtree(index2)
    ind1[slice1], ind2[slice2] = ind2[slice2], ind1[slice1]
    return ind1, ind2

def traverse_tree(stack, res, parent, idx):
    while res != 0:
        res -= 1
        idx += 1
        stack.append((parent[idx], [], idx))
        res += parent[idx].arity
    # print(f"stack: {stack}")
    return stack, res, idx

def cx_one_point(ind1, ind2):
    idx1 = 0
    idx2 = 0
    # To track the trees
    stack1 = []
    stack2 = []
    # Store the common region
    region1 = []
    region2 = []

    # Start traversing the trees
    while idx1 < len(ind1) and idx2 < len(ind2):
        # Push the nodes to the stack
        stack1.append((ind1[idx1], [], idx1))
        stack2.append((ind2[idx2], [], idx2))

        # Not the same region
        if stack1[-1][0].arity != stack2[-1][0].arity:
            res1 = stack1[-1][0].arity
            res2 = stack2[-1][0].arity
            stack1, res1, idx1 = traverse_tree(stack1, res1, ind1, idx1)
            stack2, res2, idx2 = traverse_tree(stack2, res2, ind2, idx2)
        else:
            region1.append([ind1[idx1], idx1])
            region2.append([ind2[idx2], idx2])

        idx1 += 1
        idx2 += 1

    # for pri, idx in region1:
    #     print(f"{idx}: {pri.name}")

    # Select crossover point
    if len(region1) > 0:
        point = random.randint(0, len(region1) - 1)
        # print(f"crossover point: {point}")
        # print(f"crossover point for trees: {region1[point]}, {region2[point]}")

    # Swap subtrees
    if len(region1) > 0:
        slice1 = ind1.searchSubtree(region1[point][1])
        slice2 = ind2.searchSubtree(region2[point][1])
        ind1[slice1], ind2[slice2] = ind2[slice2], ind1[slice1]

    return ind1, ind2

In [6]:
class GP():
    def __init__(self, embeddings, dimension, population_size, crossover_method, cross_prob, mut_prob, num_generations, dataset):
        self.embeddings = embeddings
        self.dim = dimension
        self.pop_size = population_size
        self.pop = None
        self.cx_method = crossover_method
        self.cx_pb = cross_prob
        self.mut_pb = mut_prob
        self.num_gen = 0
        self.inputword = dataset[0].str.split().apply(lambda x: x[:5])
        self.realword = dataset[0].str.split().str.get(5)
        self.eval_count = 0

    def register(self):
        # Function set
        self.pset = gp.PrimitiveSet("MAIN", 5)
        self.pset.addPrimitive(np.add, 2)
        self.pset.addPrimitive(np.subtract, 2)
        self.pset.addPrimitive(np.multiply, 2)
        self.pset.addPrimitive(protected_div, 2)
        self.pset.addPrimitive(protected_sqrt, 1)
        self.pset.addPrimitive(np.square, 1)
        # Terminal set
        count = 0
        self.pset.renameArguments(ARG0="a", ARG1="b", ARG2="c", ARG3="d", ARG4="e")
        # for line in self.inputword:
            # for word in line:
                # if count < 5:
                    # print(f"conut: {count}")
                    # print(f"embedding: {embeddings[word]}, {type(embeddings[word])}")
                    # count += 1
                # self.pset.addTerminal(word)

        # Initialize the individual
        if "Individual" not in creator.__dict__:
            creator.create("FitnessMax", base.Fitness, weights=(1,))
            creator.create(
                "Individual", gp.PrimitiveTree, fitness=creator.FitnessMax, pset=self.pset
            )

        # Initialize the toolbox
        self.toolbox = base.Toolbox()
        self.toolbox.register("expr", gp.genHalfAndHalf, pset=self.pset, min_=1, max_=5)
        self.toolbox.register(
            "individual", tools.initIterate, creator.Individual, self.toolbox.expr
        )
        self.toolbox.register(
            "population",
            tools.initRepeat,
            list,
            self.toolbox.individual,
            n=self.pop_size,
        )

        # Register the operators
        # self.toolbox.register("select_candidate", tools.selRandom, tournsize=3)
        self.toolbox.register("crossover", self.crossover)
        self.toolbox.register(
            "mutate", gp.mutUniform, expr=self.toolbox.expr, pset=self.pset
        )
        self.toolbox.decorate(
            "mutate", gp.staticLimit(operator.attrgetter("height"), max_value=5)
        )
        self.toolbox.register("evaluate", self.evaluate)

        # Register the record for analyzing
        self.stats = tools.Statistics(key=lambda ind: ind.fitness.values)
        self.stats.register("avg", np.mean)
        self.stats.register("std", np.std)
        self.stats.register("min", np.min)
        self.stats.register("max", np.max)
        self.hof = tools.HallOfFame(10)  # hall of fame size

    def clean_data(self, data):
        data = np.where(np.isinf(data), np.finfo(np.float32).max, data)
        data = np.nan_to_num(data, nan=0.0)
        return data

    def evaluate(self, individual, input_word):
        """Evalute the fitness of an individual"""
        # print(f"individual種類:{type(individual)}")
        func = gp.compile(individual, self.pset)
        total_similarity = 0.0
        for data_index in range(len(input_word)):
            words = self.inputword.iloc[data_index]
            in_vectors = [self.embeddings[word] for word in words]
            a, b, c, d, e = in_vectors[:5]

            y = self.realword.iloc[data_index]
            out_vector = self.embeddings[y]

            predict = self.clean_data(func(a, b, c, d, e))
            similarity = cosine_similarity([predict], [out_vector])[0][0]
            total_similarity += similarity

        fitness = total_similarity / len(self.inputword)
        ftiness = self.clean_data(fitness)
        self.eval_count += 1
        return (fitness,)

    def crossover(self, ind1, ind2):
        parents = [ind1, ind2]
        if random.uniform(0, 1) < self.cx_pb:
            if self.cx_method == CX_RANDOM:
                choice = random.randint(1, 4)
            if choice == CX_SIMPLE or self.cx_method == CX_SIMPLE:
                ind1, ind2 = cx_simple(ind1, ind2)
            elif choice == CX_UNIFORM or self.cx_method == CX_UNIFORM:
                ind1, ind2 = cx_uniform(ind1, ind2)
            elif choice == CX_ONEPOINT or self.cx_method == CX_FAIR:
                ind1, ind2 = cx_fair(ind1, ind2)
            else:  # self.cx_method == 4:
                ind1, ind2 = cx_one_point(ind1, ind2)
        # return ind1, ind2

        fitness_ind1 = self.toolbox.evaluate(ind1, self.realword)
        # ?:
        # if self.cx_method == 2:
        #     parents.remove(b)
        #     return parents
        fitness_ind2 = self.toolbox.evaluate(ind2, self.realword)
        if fitness_ind1 <= fitness_ind2:
            parents.remove(ind1)
            return parents
        else:
            parents.remove(ind2)
            return parents

    def mutate(self, offspring):
        if random.uniform(0, 1) < self.mut_pb:
            self.toolbox.mutate(offspring[0])
            offspring[0].fitness.values = self.toolbox.evaluate(offspring[0], self.realword)
        return offspring

## GPAB

### Setup

In [7]:
# Boosting parameters
boosting_interval = 20
ensemble = []

population_size = 250
dim = 10
crossover_method = CX_RANDOM
cross_prob = 1
mut_prob = 0.1
num_generations = 100
embedding = WORD2VEC

dataset, embeddings, word2vec_model = get_embeddings(WORD2VEC, 10, 1)


In [8]:
dataset.head()

,0
258316,federal election fact check underemployment sh...
210027,remote health services get staffing boost
144075,grandstand at stumps gabba day five
168439,sorenstam lands fifth major after playoff
120555,bihar flood exposes vulnerable children to


#### Initialization

In [19]:
# Initialize the population
trees = GP(embeddings, dim, population_size, crossover_method, cross_prob, mut_prob, num_generations, dataset)
trees.register()

trees.pop = trees.toolbox.population(n = trees.pop_size)
# Evaluate the entire population
fitnesses = map(trees.toolbox.evaluate, trees.pop, trees.inputword)
for ind, fit in zip(trees.pop, fitnesses):
    ind.fitness.values = fit

# Initialize instance weights
dataset["weights"] = 1.0 / len(dataset)
dataset["weights_update"] = 1.0 / len(dataset)

In [20]:
ensemble = []
# Start the evolution
while trees.num_gen < num_generations:
    # Parent selection
    candidates = tools.selRandom(trees.pop, 3)
    sorted_candidates = sorted(candidates, key=lambda x: x.fitness.values)  # Small to large
    # print(f"sorted_candidates: {[ind.fitness.values for ind in sorted_candidates]}")

    # Crossover
    parent1, parent2 = copy.deepcopy(candidates[0]), copy.deepcopy(candidates[1])
    offspring = trees.toolbox.crossover(parent1, parent2)

    # Mutation
    offspring = trees.toolbox.mutate(offspring[0])

    # Survival selection
    offspring[0].fitness.values = trees.toolbox.evaluate(offspring[0], trees.realword)
    if offspring[0].fitness.values > sorted_candidates[2].fitness.values:
        idx = trees.pop.index(sorted_candidates[2])
        trees.pop[idx] = offspring[0]

    if trees.num_gen % boosting_interval == 0:
        print(f"Boosting at iteration {trees.num_gen}")
        record = trees.stats.compile(trees.pop)
        trees.hof.update(trees.pop)
        print(record)
        # total_weights = np.sum(dataset["weights"])
        errors = []
        for data_index in range(len(dataset)):
            func = gp.compile(ind, trees.pset)
            a, b, c, d, e = [trees.embeddings[word] for word in trees.inputword.iloc[1]]
            y_pred = (func(a, b, c, d, e))
            y_true = (trees.embeddings[trees.realword.iloc[data_index]])

            # Calculate the prediction error (e.g., Euclidean distance)
            error = np.linalg.norm(y_true - y_pred)
            # print(f"error: {error}")
            errors.append(error)
        # print(f"errors: {len(errors)}")

        # Normalize errors
        max_error = max(errors)
        if max_error == 0:
            max_error = 1
        errors = [error / max_error for error in errors]




        # Get best individual
        best = max(trees.pop, key=lambda x: x.fitness.values)
        epsilon = 1 - best.fitness.values[0]
        best_weight = np.log((1 - epsilon) / epsilon)
        ensemble.append(best)
        new_weight = np.exp(best_weight)
        print(f"new_weight: {new_weight}")





        # Update instance weights
        dataset["weights_update"] *= np.exp(errors) * new_weight
        # Normalize weights
        sum_weights = np.sum(dataset["weights_update"])
        dataset["weights_update"] = dataset["weights_update"] / sum_weights

        # Update the population dataset
        select_new_data = np.random.uniform(0, 1, len(dataset))
        dataset["cumulative_weights"] = dataset["weights_update"].cumsum()
        # Find the indices of the closest rows in cumulative_weights for each value in select_new_data
        indices = np.digitize(select_new_data, dataset["cumulative_weights"])
        # Create new dataset by selecting rows from original dataset based on indices
        new_dataset = dataset.iloc[indices].reset_index(drop=True)

        trees.dataset = new_dataset
        # Evaluate the entire population
        fitnesses = map(trees.toolbox.evaluate, trees.pop, trees.inputword)
        for ind, fit in zip(trees.pop, fitnesses):
            ind.fitness.values = fit

    # Show the record
    if trees.eval_count % 50 == 0:
        print(f"Evaloation {trees.eval_count}")
        record = trees.stats.compile(trees.pop)
        trees.hof.update(trees.pop)
        print(record)
    trees.num_gen += 1

Boosting at iteration 0
{'avg': 0.0008414324622615817, 'std': 0.010877532362023977, 'min': -0.0008253868586921478, 'max': 0.17230871923071736}
new_weight: 0.20817993765811826
Boosting at iteration 20
{'avg': 0.002688088464772753, 'std': 0.021076442611122865, 'min': -0.0008253868586921478, 'max': 0.2784130670258903}
new_weight: 0.3858344078909196
Boosting at iteration 40
{'avg': 0.008985940092153791, 'std': 0.053407719922331134, 'min': -0.0008253868586921478, 'max': 0.42328820737812783}
new_weight: 0.7339683578408494
Evaloation 1150
{'avg': 0.003943132394234313, 'std': 0.033369355727966746, 'min': -0.0008253868586921478, 'max': 0.40309313294350746}
Boosting at iteration 60
{'avg': 0.007751041929401153, 'std': 0.046627464304589174, 'min': -0.0008253868586921478, 'max': 0.40309313294350746}
new_weight: 0.6753032259978303
Boosting at iteration 80
{'avg': 0.0048788658784267155, 'std': 0.02905386484506769, 'min': -0.0008253868586921478, 'max': 0.3066233919588794}
new_weight: 0.44221767565123

In [11]:
record = trees.stats.compile(trees.pop)
trees.hof.update(trees.pop)
print(record)

{'avg': 0.008515772810846471, 'std': 0.04964755771014911, 'min': -0.0008674927723010232, 'max': 0.40897962526391574}


### Try

In [71]:
dataset.head()

,0,weights,weights_update,cumulative_weights
142305,contracts being finalised for gold mine,0.000375,0.000375,0.000375
133777,sa naplan results behind national average,0.000375,0.000375,0.000749
10775,captain detained over bahrain boat accident,0.000375,0.000375,0.001124
214874,gladstone ports corporation to boost jobs,0.000375,0.000375,0.001499
139083,messi wins world cup golden ball,0.000375,0.000375,0.001873


In [65]:
select_new_data = np.random.uniform(0, 1, len(dataset))
dataset["cumulative_weights"] = dataset["weights_update"].cumsum()


In [78]:
# Ensure cumulative_weights is sorted
# dataset.sort_values("cumulative_weights", inplace=True)

# Find the indices of the closest rows in cumulative_weights for each value in select_new_data
indices = np.digitize(select_new_data, dataset["cumulative_weights"])

# Create new dataset by selecting rows from original dataset based on indices
new_dataset = dataset.iloc[indices].reset_index(drop=True)

In [79]:
new_dataset

,0,weights,weights_update,cumulative_weights
0,nsw premier accepts mike gallachers resignation,0.000375,0.000375,0.100037
1,advisor resigns over conflict of interest,0.000375,0.000375,0.917197
2,broke residents call for blast action,0.000375,0.000375,0.440614
3,funeral service held for philippines landslide,0.000375,0.000375,0.306107
4,nationals call for bushfire evacuation database,0.000375,0.000375,0.368303
...,...,...,...,...
2664,crews to begin south coast backburning,0.000375,0.000375,0.858749
2665,the fight for the kitchen table,0.000375,0.000375,0.480704
2666,police investigate ute rampage northern victoria,0.000375,0.000375,0.619333
2667,man pleads guilty over child porn,0.000375,0.000375,0.217310


In [80]:
new_dataset.duplicated().sum()

973

In [26]:
# Get best individual
best = max(trees.pop, key=lambda x: x.fitness.values)
epsilon = 1 - best.fitness.values[0]
best_weight = np.log((1 - epsilon) / epsilon)
ensemble.append(best)
new_weight = np.exp(best_weight)
print(f"new_weight: {new_weight}")

new_weight: 0.0008214122571255709


In [36]:
dataset.head()

,0,weights
23729,council concerned about paringa riverbank erosion,0.000375
76530,controversial russian artist disappears in berlin,0.000375
257892,simplot increases prices for pea growers,0.000375
93799,pacific national train drivers cancel planned,0.000375
237328,thousands of south australians stranded waiting,0.000375


In [ ]:
# Get all the outputs of the GP trees
y_pred = []
y_true = []
for ind in trees.pop:
    func = gp.compile(ind, trees.pset)
    a, b, c, d, e = [trees.embeddings[word] for word in trees.inputword.iloc[1]]
    y_pred.append(func(a, b, c, d, e))
    y_true.append(trees.embeddings[trees.realword.iloc[data_index]])

In [15]:
# Predict using the best individual from the current population
best_individual = tools.selBest(trees.pop, 1)[0]
func = gp.compile(best_individual, trees.pset)
a, b, c, d, e = [trees.embeddings[word] for word in trees.inputword.iloc[1]]
predict = trees.clean_data(func(a, b, c, d, e))
predict = predict / np.linalg.norm(predict) if np.linalg.norm(predict) != 0 else predict

In [37]:
# def update_instance_weights(trees, dataset):
total_weights = np.sum(dataset["weights"])
errors = []

for data_index in range(len(dataset)):
    func = gp.compile(ind, trees.pset)
    a, b, c, d, e = [trees.embeddings[word] for word in trees.inputword.iloc[1]]
    y_pred = (func(a, b, c, d, e))
    y_true = (trees.embeddings[trees.realword.iloc[data_index]])

    # Calculate the prediction error (e.g., Euclidean distance)
    error = np.linalg.norm(y_true - y_pred)
    # print(f"error: {error}")
    errors.append(error)
errors[:5]

[34.80443, 35.789825, 37.67824, 36.81569, 38.063366]

In [38]:
# Normalize errors
max_error = max(errors)
if max_error == 0:
    max_error = 1
errors = [error / max_error for error in errors]
errors[:5]

[0.79609025, 0.81862944, 0.8618237, 0.8420943, 0.8706327]

In [40]:
np.exp(errors)

array([2.2168567, 2.26739  , 2.3674743, ..., 2.334882 , 2.2394364,
       2.3770418], dtype=float32)

In [41]:
# Update weights
for i in range(len(dataset)):
    dataset['weights_update'] = dataset["weights"] * np.exp(errors)

In [42]:
dataset.head()

,0,weights,weights_update
23729,council concerned about paringa riverbank erosion,0.000375,0.000831
76530,controversial russian artist disappears in berlin,0.000375,0.000850
257892,simplot increases prices for pea growers,0.000375,0.000887
93799,pacific national train drivers cancel planned,0.000375,0.000870
237328,thousands of south australians stranded waiting,0.000375,0.000895


In [43]:
# Normalize weights
sum_weights = np.sum(dataset["weights_update"])
sum_weights

2.344232827726488

In [44]:
dataset["weights_update"] /= sum_weights

In [45]:
dataset.head()

,0,weights,weights_update
23729,council concerned about paringa riverbank erosion,0.000375,0.000354
76530,controversial russian artist disappears in berlin,0.000375,0.000362
257892,simplot increases prices for pea growers,0.000375,0.000378
93799,pacific national train drivers cancel planned,0.000375,0.000371
237328,thousands of south australians stranded waiting,0.000375,0.000382
